In [2]:
!nvidia-smi

Wed Dec 23 09:35:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    22W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html


     |████████████████████████████████| 55.0MB 101kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 15.5MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588514 sha256=df29a3f965903e95f06eb413fe6fa7428cfc0f13a63dbb08833f0c48251c35ee
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 12.8MB/s 
     |████████████████████████████████| 757kB 13.4MB/s 
     |████████████████████████████████| 890kB 29.5MB/s 
     |████████████████████████████████| 3.0MB 49.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=de8244e1b4c04fe9ff2bd3172370f699a04fc10dc3c542eefde46a8ca5ee95df
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a

In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-xi38n5t8
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-xi38n5t8
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=27b498166bde6a8cd3d9c31b86eaae50fd8be4905794cc5a6afc01ece87f0745
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvttx1bp/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [7]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
##GPU 사용 시
device = torch.device("cuda:0")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [10]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2020-12-23 09:39:01--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2020-12-23 09:39:02--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc355f1874c51c15ceb60106cf0e.dl.dropboxusercontent.com/cd/0/get/BFl-3ICEM9HqPfVHZTCGZzTyaKp1C5AYikP4oYU1-dujfvf9NZHNqHNN7gnS7aWX55qEcfRy3DSnSyIlHn9MrROtiocAstdt51pc4BB5XUb2MtdP7O7yj1lxcqmSocisQxM/file?dl=1# [following]
--2020-12-23 09:39:02--  https://uc355f1874c51c15ceb60106cf0e.dl.dropboxusercontent.com/cd/0/get/BFl-3ICEM9HqPfVHZTCGZzTyaKp1C5AYikP4oYU1-dujfvf9NZHNqHNN7gnS7aWX55qEcfRy3DSnSyIlHn9MrROtioc

In [11]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [12]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [14]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [15]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [16]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [17]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [18]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [19]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [20]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [21]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [22]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [23]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [24]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.7459260821342468 train acc 0.4375
epoch 1 batch id 201 loss 0.6029626131057739 train acc 0.5316386815920398
epoch 1 batch id 401 loss 0.4698720872402191 train acc 0.6491583541147132
epoch 1 batch id 601 loss 0.38720521330833435 train acc 0.710196547420965
epoch 1 batch id 801 loss 0.4071270227432251 train acc 0.7433871722846442
epoch 1 batch id 1001 loss 0.31709811091423035 train acc 0.7639235764235764
epoch 1 batch id 1201 loss 0.39500775933265686 train acc 0.7796107410491258
epoch 1 batch id 1401 loss 0.40155109763145447 train acc 0.7902614204139901
epoch 1 batch id 1601 loss 0.31369590759277344 train acc 0.79970526233604
epoch 1 batch id 1801 loss 0.25957047939300537 train acc 0.8070082593003887
epoch 1 batch id 2001 loss 0.3152817189693451 train acc 0.8135619690154923
epoch 1 batch id 2201 loss 0.3088349401950836 train acc 0.8190524193548387

epoch 1 train acc 0.8229611063708759


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8865688938618926


epoch 2 batch id 1 loss 0.4701389968395233 train acc 0.796875
epoch 2 batch id 201 loss 0.23303405940532684 train acc 0.8777207711442786
epoch 2 batch id 401 loss 0.26011237502098083 train acc 0.8816630299251871
epoch 2 batch id 601 loss 0.3737535774707794 train acc 0.8860232945091514
epoch 2 batch id 801 loss 0.3714130222797394 train acc 0.8883817103620475
epoch 2 batch id 1001 loss 0.2573486268520355 train acc 0.8908123126873126
epoch 2 batch id 1201 loss 0.1882207989692688 train acc 0.8929407785179018
epoch 2 batch id 1401 loss 0.21076567471027374 train acc 0.8948518915060671
epoch 2 batch id 1601 loss 0.2114163041114807 train acc 0.8968222985633979
epoch 2 batch id 1801 loss 0.17238053679466248 train acc 0.8987107856746253
epoch 2 batch id 2001 loss 0.23012661933898926 train acc 0.9006278110944528
epoch 2 batch id 2201 loss 0.20456059277057648 train acc 0.9022248409813721

epoch 2 train acc 0.9036036156143344


...


epoch 3 test acc 0.8967191496163683


epoch 4 batch id 1 loss 0.3208966851234436 train acc 0.875
epoch 4 batch id 201 loss 0.05568578466773033 train acc 0.9567008706467661
epoch 4 batch id 401 loss 0.05572352185845375 train acc 0.9566708229426434
epoch 4 batch id 601 loss 0.15411509573459625 train acc 0.9585326539101497
epoch 4 batch id 801 loss 0.1686546802520752 train acc 0.9600499375780275
epoch 4 batch id 1001 loss 0.0663103237748146 train acc 0.960836038961039
epoch 4 batch id 1201 loss 0.02487349696457386 train acc 0.9621929641965029
epoch 4 batch id 1401 loss 0.02796880342066288 train acc 0.963006334760885
epoch 4 batch id 1601 loss 0.12454298883676529 train acc 0.9635481730168645
epoch 4 batch id 1801 loss 0.09966892004013062 train acc 0.964585646862854
epoch 4 batch id 2001 loss 0.03969745710492134 train acc 0.9653220264867566
epoch 4 batch id 2201 loss 0.06238550320267677 train acc 0.9658535892776011

epoch 4 train acc 0.9664324694254836



epoch 4 test acc 0.8966791879795396


epoch 5 batch id 1 loss 0.4395920932292938 train acc 0.890625
epoch 5 batch id 201 loss 0.029274161905050278 train acc 0.9718594527363185
epoch 5 batch id 401 loss 0.023722201585769653 train acc 0.9754130299251871
epoch 5 batch id 601 loss 0.16025874018669128 train acc 0.9757955490848585
epoch 5 batch id 801 loss 0.10387659817934036 train acc 0.9759285268414482
epoch 5 batch id 1001 loss 0.034890104085206985 train acc 0.976663961038961
epoch 5 batch id 1201 loss 0.017576152458786964 train acc 0.9773626144879267
epoch 5 batch id 1401 loss 0.021924879401922226 train acc 0.9774826017130621
epoch 5 batch id 1601 loss 0.056922223418951035 train acc 0.9776897251717677
epoch 5 batch id 1801 loss 0.10487355291843414 train acc 0.978241254858412
epoch 5 batch id 2001 loss 0.027893399819731712 train acc 0.9784560844577711
epoch 5 batch id 2201 loss 0.036559268832206726 train acc 0.9785892776010904

epoch 5 train acc 0.9788822525597269



epoch 5 test acc 0.897878037084399


In [25]:
from google.colab import drive
isMount=!df |grep /content/drive
if len(isMount) < 1 : drive.mount('/content/drive')


In [26]:
# Model Save
torch.save(model.state_dict(), "/content/drive/MyDrive/myBertModel")


In [27]:
!ls /content/drive/MyDrive/myBertModel

/content/drive/MyDrive/myBertModel


In [28]:
!head ratings_te*


id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??	0
7898805	음악이 주가 된, 최고의 음악영화	1
6315043	진정한 쓰레기	0
6097171	마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다	0
8932678	갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다	0
